In [2]:
import numpy as np
import tensorflow as tf

# 07-03: Convolution Operation Naive Backward Pass

Try calculating NumPy, then check with TensorFlow and numerical gradient

#### 1. Basic
$(4 \times 4) * (3 \times 3) = (2 \times 2)$

#### 2. Padding
$(4 \times 4) * (3 \times 3) = (4 \times 4)$ where $P=1$

#### 3. Stride
$(7 \times 7) * (3 \times 3) = (3 \times 3)$ where $S=2$

#### 4. Padding and Stride
$(7 \times 7) * (3 \times 3) = (4 \times 4)$ where $P=1, S=2$

#### 5. Channel
$(4 \times 4 \times 3) * (3 \times 3 \times 3) = (2 \times 2)$

#### 6. Channel and bias 
$(4 \times 4 \times 3) * (3 \times 3 \times 3) + (1) = (2 \times 2)$

#### 7. Multiple Filters
$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) = (2 \times 2 \times 4)$

#### 8.Multiple Filters + bias 
$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (2 \times 2 \times 4)$

#### 9. Mini-batch + bias
$(3 \times 4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 2 \times 2 \times 4)$

#### 10. RGB Mini-batch $*$ Multiple Filters with stride and padding
$(3 \times 7 \times 7 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 4 \times 4 \times 4)$ where $P=1, S=2$


In [3]:
def float_sequence(size):
    return np.arange(size, dtype=np.float32)

### 1. Basic Convolution

$(4 \times 4) * (3 \times 3) = (2 \times 2)$

In [29]:
X = float_sequence(4*4).reshape(4,4)
W = 12 - float_sequence(3*3).reshape(3,3)

dY = np.ones((2,2))

print("=== dY ===")     
print(dY)

dX = np.zeros((4,4))
dW = np.zeros((3,3))

for h in range(4-3+1):
    for w in range(4-3+1):
        h_start = h
        h_end   = h_start + 3
        w_start = w
        w_end   = w_start + 3
        
        current_dY = dY[h, w]
        dX[h_start:h_end, w_start:w_end] += current_dY * W
        dW += current_dY * X[h_start:h_end, w_start:w_end]
        
print("=== dX ===")     
print(dX)
        
print("=== dW ===")     
print(dW)

with tf.Session() as sess:
    tf_X = tf.constant(X.reshape(1, 4, 4, 1))
    tf_W = tf.Variable(W.reshape(3, 3, 1, 1))
    tf_Y = tf.nn.conv2d(tf_X, tf_W, strides=[1, 1, 1, 1], padding='VALID')
    tf_L = tf.reduce_sum(tf_Y)
    tf_grad = tf.gradients(tf_L, [tf_X, tf_W])
    
    sess.run(tf.global_variables_initializer())
    tf_L_val = sess.run(tf_L)
    tf_grad_val = sess.run(tf_grad)
    print("=== L (tf) ===")     
    print(tf_L_val)
    print("=== dX (tf) ===")     
    print(tf_grad_val[0][0, :, :, 0])
    print("=== dW (tf) ===")     
    print(tf_grad_val[1][:, :, 0, 0])

print("=== Matched? ===")    
print("dX: ", np.all(dX == tf_grad_val[0][0, :, :, 0]))
print("dY: ", np.all(dW == tf_grad_val[1][:, :, 0, 0]))

=== dY ===
[[ 1.  1.]
 [ 1.  1.]]
=== dX ===
[[ 12.  23.  21.  10.]
 [ 21.  40.  36.  17.]
 [ 15.  28.  24.  11.]
 [  6.  11.   9.   4.]]
=== dW ===
[[ 10.  14.  18.]
 [ 26.  30.  34.]
 [ 42.  46.  50.]]
=== L (tf) ===
1848.0
=== dX (tf) ===
[[ 12.  23.  21.  10.]
 [ 21.  40.  36.  17.]
 [ 15.  28.  24.  11.]
 [  6.  11.   9.   4.]]
=== dW (tf) ===
[[ 10.  14.  18.]
 [ 26.  30.  34.]
 [ 42.  46.  50.]]
=== Matched? ===
dX:  True
dY:  True


### 2. Convolution with padding

$(4 \times 4) * (3 \times 3) = (4 \times 4)$ where $P=1$

### 3. Convolution with Stride

$(7 \times 7) * (3 \times 3) = (3 \times 3)$ where $S=2$

### 4. Padding and Stride

$(7 \times 7) * (3 \times 3) = (4 \times 4)$ where $P=1, S=2$

### 5. Channel

$(4 \times 4 \times 3) * (3 \times 3 \times 3) = (2 \times 2)$

### 6. Channel and bias 

$(4 \times 4 \times 3) * (3 \times 3 \times 3) + (1) = (2 \times 2)$

### 7. Multiple Filters

$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) = (2 \times 2 \times 4)$

### 8. Multiple Filters + bias 

$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (2 \times 2 \times 4)$

### 9. Mini-batch + bias

$(3 \times 4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 2 \times 2 \times 4)$

#### 10. RGB Mini-batch $*$ Multiple Filters with stride and padding
$(3 \times 7 \times 7 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 4 \times 4 \times 4)$ where $P=1, S=2$